In [76]:
# Remember: library imports are ALWAYS at the top of the script, no exceptions!
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
import warnings
warnings.filterwarnings('ignore')
import dtale

from itertools import product
from pandas_profiling import ProfileReport


%config InlineBackend.figure_format = 'retina' # optionally, you can change 'svg' to 'retina'

# Seeting seaborn style
sns.set()


from datetime import datetime
import calendar

In [57]:
data = pd.read_csv ("/Users/vsa/Documents/DM/data/donors.csv", low_memory=False)

In [48]:
dtale.show(data, ignore_duplicate=True)

In [8]:

# df.DOB  = df.DOB.map(lambda x: datetime.strptime(x, '%Y-%m-%d'))
# df['YOB']  = df.DOB.map(lambda x: x.year)

## Calculating age, recency etc

In [58]:
# Date of donor's first gift to PVA
# cannot be earlier than his/her Date of Birthday (DOB)
len(data[pd.to_datetime(data.ODATEDW) <= pd.to_datetime(data.DOB)])

318

In [59]:
# AGE
# Here I calculate age as it was discussed during Q&A session:
# Calculate age on the moment of ADATE_2, paying attention that ADATE_2 has 2 different values in a column:
# 2017-06-01 (95399 instances), 2017-04-01 (13 instances)
data.DOB = pd.to_datetime(data.DOB)
data.ADATE_2 = pd.to_datetime(data.ADATE_2)
data["AGE"] = ((data.ADATE_2) - (data.DOB)).astype('timedelta64[Y]') #.astype('int')
#data.AGE = data.AGE.where(data.AGE != 0, np.nan)
data.AGE

0        59.0
1        45.0
2         NaN
3        69.0
4        77.0
         ... 
95407     NaN
95408    47.0
95409    59.0
95410    57.0
95411    79.0
Name: AGE, Length: 95412, dtype: float64

In [ ]:
# Next I will calculate Recency code for RFA_2
#                                     F=FIRST TIME DONOR Anyone who has made their
#                                       first donation in the last 6 months and has
#                                       made just one donation.
    
#                                     N=NEW DONOR Anyone who has made their first
#                                       donation in the last 12 months and is not a
#                                       First time donor.  This is everyone who made
#                                       their first donation 7-12 months ago, or
#                                       people who made their first donation between
#                                       0-6 months ago and have made 2 or more
#                                       donations.
    
#                                     A=ACTIVE DONOR Anyone who made their first
#                                       donation more than 12 months ago and has made
#                                       a donation in the last 12 months.
    
#                                     L=LAPSING DONOR A previous donor who made their
#                                       last donation between 13-24 months ago.
    
#                                     I=INACTIVE DONOR A previous donor who has not
#                                       made a donation in the last 24 months.  It is
#                                       people who made a donation 25+ months ago.
    
#                                     S=STAR DONOR STAR Donors are individuals who
#                                       have given to 3 consecutive card mailings.


PER TASK: One group that is of particular interest to PVA is "Lapsed" donors. These are individuals who made their last donation to PVA 13 to 24 months ago. 

Let's find out all these groups of donors and create 'RECENCY' column

In [100]:
# Calculate amount of months from the last day LASTDATE (Date associated with the most recent gift)
# and ADATE_2 - Date the latest promotion was sent
data ['ADATE_MONTHS'] = (pd.to_datetime(data.ADATE_2) - pd.to_datetime(data.LASTDATE)).astype('timedelta64[M]') 

data['RECENCY']= ''
# Print Lapsed donours (that made their last donation to PVA 13 to 24 months ago )
print(len(data [(data ['ADATE_MONTHS'] >=13.0)& (data ['ADATE_MONTHS'] <=24.0)]))
data.loc[(data['ADATE_MONTHS'] >= 13.0)& (data['ADATE_MONTHS'] <= 24.0), 'RECENCY'] = 'L'

84132


In [101]:
# Convert ODATEDW to datetime
data.ODATEDW = pd.to_datetime(data.ODATEDW)
data ['ODATEDW_MONTHS'] = (pd.to_datetime(data.ADATE_2) - pd.to_datetime(data.ODATEDW)).astype('timedelta64[M]')
#  Print number of FIRST TIME DONOR donours (F=FIRST TIME DONOR Anyone who has made their first donation 
# in the last 6 months and has and has made just one donation.)

print(len(data [(data['ADATE_MONTHS'] <= 6.0) & ((data['NGIFTALL'] == 1))]))
data.loc[(data['ADATE_MONTHS'] <= 6.0) & (data['NGIFTALL'] == 1), 'RECENCY'] = 'F'

12


In [ ]:
# N = NEW DONOR Anyone who has made their first donation in the last 12 months and is not a First time donor.  
# This is everyone who made their first donation 7-12 months ago, or
# people who made their first donation between 0-6 months ago and have made 2 or more donations.

In [66]:
# Checking for inconsistencies between ADATE_MONTHS and ODATEDW_MONTHS
print(len(data [data ['ADATE_MONTHS'] > data['ODATEDW_MONTHS']]))
print ('Percent of inconsistencies ', 3627/95412*100)

#Percent is to high to delete all these rows

3627
Percent of inconsistencies  3.8014086278455537


##### CHECKING FOR INCONSISTENCIES IN TCODE

In [13]:
dictTCODE = {
    "0" : "_",
    "1" : "MR.",
    "1001" : "MESSRS.",
    "1002" : "MR. & MRS.",
    "2" : "MRS.",
    "2002" : "MESDAMES",
    "3" : "MISS",
    "3003" : "MISSES",
    "4" : "DR.",
    "4002" : "DR. & MRS.",
    "4004" : "DOCTORS",
    "5" : "MADAME",
    "6" : "SERGEANT",
    "9" : "RABBI",
    "10" : "PROFESSOR",
    "10002" : "PROFESSOR & MRS.",
    "10010" : "PROFESSORS",
    "11" : "ADMIRAL",
    "11002" : "ADMIRAL & MRS.",
    "12" : "GENERAL",
    "12002" : "GENERAL & MRS.",
    "13" : "COLONEL",
    "13002" : "COLONEL & MRS.",
    "14" : "CAPTAIN",
    "14002" : "CAPTAIN & MRS.",
    "15" : "COMMANDER",
    "15002" : "COMMANDER & MRS.",
    "16" : "DEAN",
    "17" : "JUDGE",
    "17002" : "JUDGE & MRS.",
    "18" : "MAJOR",
    "18002" : "MAJOR & MRS.",
    "19" : "SENATOR",
    "20" : "GOVERNOR",
    "21002" : "SERGEANT & MRS.",
    "22002" : "COLONEL & MRS.", #mistype in COLONEL, was changed here
    "24002" : "LIEUTENANT & MRS.", #XX002 code is used for someone & MRS.
    "24" : "LIEUTENANT",
    "26" : "MONSIGNOR",
    "27" : "REVEREND",
    "28" : "MS.",
    "28028" : "MSS.",
    "29" : "BISHOP",
    "31" : "AMBASSADOR",
    "31002" : "AMBASSADOR & MRS.",
    "33" : "CANTOR",
    "36" : "BROTHER",
    "37" : "SIR",
    "38" : "COMMODORE",
    "40" : "FATHER",
    "42" : "SISTER",
    "43" : "PRESIDENT",
    "44" : "MASTER",
    "46" : "MOTHER",
    "47" : "CHAPLAIN",
    "48" : "CORPORAL",
    "50" : "ELDER",
    "56" : "MAYOR",
    "59002" : "LIEUTENANT & MRS.",
    "62" : "LORD",
    "63" : "CARDINAL",
    "64" : "FRIEND",
    "65" : "FRIENDS",
    "68" : "ARCHDEACON",
    "69" : "CANON",
    "70" : "BISHOP",
    "72002" : "REVEREND & MRS.",
    "73" : "PASTOR",
    "75" : "ARCHBISHOP",
    "85" : "SPECIALIST",
    "87" : "PRIVATE",
    "89" : "SEAMAN", 
    "90" : "AIRMAN",
    "91" : "JUSTICE",
    "92" : "MR. JUSTICE",
    "100" : "M.",
    "103" : "MLLE.",
    "104" : "CHANCELLOR",
    "106" : "REPRESENTATIVE",
    "107" : "SECRETARY",
    "108" : "LT. GOVERNOR",
    "109" : "LIC.",
    "111" : "SA.",
    "114" : "DA.",
    "116" : "MR.", #Changed SR ro MR
    "117" : "MRS.", #Changed SRA ro MRS
    "118" : "MISS", #Cnanged SRTA. to MISS
    "120" : "YOUR MAJESTY",
    "122" : "HIS HIGHNESS",
    "123" : "HER HIGHNESS",
    "124" : "COUNT",
    "125" : "LADY",
    "126" : "PRINCE",
    "127" : "PRINCESS",
    "128" : "CHIEF",
    "129" : "BARON",
    "130" : "SHEIK",
    "131" : "PRINCE AND PRINCESS",
    "132" : "YOUR IMPERIAL MAJEST",
    "135" : "M. ET MME.",
    "210" : "PROF.",
    '72': 'REVEREND', #because code 72002 REVEREND AND MRS, I think it was 072 and 027 for REVEREND 
    #because some systematic mistake (mistype), and in the end someone changed it in system
    '22': "COLONEL & MRS.", #just a mistype,
    
}

In [14]:
data['TCODE'] = data['TCODE'].apply(str)

data["cat_TCODE"] = data.TCODE.replace(to_replace=dictTCODE)

CHECKING FOR INCONSISTENCIES IN TCODE
- wrote down in doc all summary

In [28]:
# CHECKING FOR INCONSISTENCIES IN TCODE
print('Number of Men(GENDER M) with title MRS.(Mrs - womans title of married woman)', 
      len(data[(data.GENDER == 'M')&(data.cat_TCODE == 'MRS.')]))
print('Number of men with title MISS (title of young or unmarried woman)', 
      len(data[(data.GENDER == 'M')&(data.cat_TCODE == 'MISS')]))
print('Number of men with title MS. (title of married or not married woman)', 
      len(data[(data.GENDER == 'M')&(data.cat_TCODE == 'MS.')]))
print('Number of women with title MR. (stays for mister, mans title)', 
      len(data[(data.GENDER == 'F')&(data.cat_TCODE == 'MR.')]))
print('Number of men with title Sister', 
      len(data[(data.GENDER == 'M')&(data.cat_TCODE == 'SISTER')]))
print('Number of women with title Brother', 
      len(data[(data.GENDER == 'F')&(data.cat_TCODE == 'BROTHER')]))
#women (F) that has code 'MR. & MRS.' that implies that they should be me
print('Number of women with title MR. & MRS., that implies that they should be men',
      len(data[(data.GENDER == 'F')&(data.cat_TCODE == 'MR. & MRS.')]))

Number of Men(GENDER M) with title MRS.(Mrs - womans title of married woman) 3412
Number of men with title MISS (title of young or unmarried woman) 34
Number of men with title MS. (title of married or not married woman) 216
Number of women with title MR. (stays for mister, mans title) 255
Number of men with title Sister 1
Number of women with title Brother 0
Number of women with title MR. & MRS., that implies that they should be men 46
